# 0. Import Dependencies

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
with open("./traffic-signs-data/train.p", mode = 'rb') as train_data:
    train = pickle.load(train_data)
with open("./traffic-signs-data/train.p", mode = 'rb') as test_data:
    test = pickle.load(test_data)
with open("./traffic-signs-data/train.p", mode = 'rb') as valida_data:
    valid = pickle.load(valida_data)